1/14 최초 구현 by 소연 

수정 및 테스트 시 본 파일이 아닌 사본 사용을 부탁드립니다.


1/18  수정 by 지현

* 회의록 음성 파일 일정 크기로 자르는 loop추가 --> 회의 전체 내용이 chked에 들어가도록 하였음
* 음성 파일 중 일부(2개) .UnknownValueError 발생하여 제거 --> 제거한 채로 갈지, 따로 처리해줘야 할지?

1/18 수정 by 소연

* UnknownValueError 발생하는 이슈 해결
* STT loop 함수화 (wav ~ spell check)
* audio_only_2.m4a -> 14일자 회의 녹음본!
* 새로운 문제: 이제 txt(or chked) 길이가 너무 길어져서 기존 bert api가 summarize 못함(글자수 제한) -> 새로운 api/model 찾아주세요오

(파일이 길어져서 굳이 볼필요없는 코드는 숨겼는데 확장하려면 각 타이틀 더블클릭하면 됩니당!)

In [ ]:
#@title 실행 환경 구축
import os, sys
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/Shareddrives/KPMG_Ideation
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from pprint import pprint
from krwordrank.word import KRWordRank
from krwordrank.sentence import summarize_with_sentences
from copy import deepcopy
import kss
import itertools
import unicodedata
import requests
from functools import reduce
from bs4 import BeautifulSoup
from gensim.summarization.summarizer import summarize
import string
import torch
from textrankr import TextRank
from lexrankr import LexRank
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize, sent_tokenize 
from pydub import AudioSegment
from konlpy.tag import Okt
import re
import nltk
nltk.download('punkt')

# import pre-trained model -- frameBERT (pytorch GPU 환경 필요)
%cd /content/drive/Shareddrives/KPMG_Ideation/OpenInformationExtraction/frameBERT
!pip install transformers 
import frame_parser
path="/content/drive/Shareddrives/KPMG_Ideation/OpenInformationExtraction/frameBERT"
parser = frame_parser.FrameParser(model_path=path, language='ko')

# Due to google api credentials, SpeechRecognition needs to be installed everytime
!pip install SpeechRecognition
import speech_recognition as sr
#!pip install google-api-core==1.22.2
# !pip install --upgrade google-api-core
# !pip install --upgrade google-cloud-speech

##### below are permanently installed packages #####
# nb_path = '/content/notebooks'
# os.symlink('/content/drive/Shareddrives/KPMG_Ideation', nb_path)
# sys.path.insert(0, nb_path)
# !pip install --target=$nb_path pydub
# !pip install --target=$nb_path kss
# %cd /content/drive/Shareddrives/KPMG_Ideation/hanspell
# !python setup.py install
# !pip install --target=$nb_path transformers
# !apt-get update
# !apt-get g++ openjdk-8-jdk 
# !pip3 install --target=$nb_path konlpy
# !pip install --target=$nb_path soykeyword
# !pip install --target=$nb_path krwordrank
# !pip install --target=$nb_path bert
# !pip install --target=$nb_path textrankr
# !pip install --target=$nb_path lexrankr

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/Shareddrives/KPMG_Ideation
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
#@title Speech-To-Text 모듈

%cd /content/drive/Shareddrives/KPMG_Ideation/hanspell
from hanspell import spell_checker

def STT_loop(file_name, startMin = 0, startSec = 0, endMin = 4, endSec = 30):
    if file_name.split('.')[1] == 'mp3':    
        sound = AudioSegment.from_mp3(file_name)
        wav_filename = file_name.split('.')[0] + '.wav'
        file_handle = sound.export(file_name, format="wav")
        song = AudioSegment.from_wav(wav_filename)
    if file_name.split('.')[1] == 'm4a':
        track = AudioSegment.from_file(file_name,'m4a')
        wav_filename = file_name.replace('m4a', 'wav')
        file_handle = track.export(wav_filename, format='wav') 
        song = AudioSegment.from_wav(wav_filename)
    
    # Time to miliseconds
    startTime = startMin*60*1000+startSec*1000
    endTime = endMin*60*1000+endSec*1000

    num_frac=int(len(song)/endTime)
    for i in range(num_frac+1):
      extract = song[startTime:min(endTime,len(song))]
      extract.export('./audio_frac/frac_'+str(i)+'.wav', format="wav")
      startTime=endTime
      endTime+=endMin*60*1000+endSec*1000

    # Speech Recognition Loop
    txt=''
    for i in range(num_frac+1):
      AUDIO_FILE = os.path.join(os.path.dirname(os.path.abspath('data')), "audio_frac/frac_"+str(i)+".wav")

      # use the audio file as the audio source
      r = sr.Recognizer()
      with sr.AudioFile(AUDIO_FILE) as source:
          audio = r.record(source)  # read the entire audio file

      # recognize speech using Google Speech Recognition
      try:
          # for testing purposes, we're just using the default API key
          # to use another API key, use `r.recognize_google(audio, key="GOOGLE_SPEECH_RECOGNITION_API_KEY")`
          # instead of `r.recognize_google(audio)`
          txt += ' '+ r.recognize_google(audio, language='ko')
          #print(str(i)+"/"+str(num_frac+1)+" done")
      except sr.UnknownValueError:
          #print("Google Speech Recognition could not understand audio")
          pass
      except sr.RequestError as e:
          #print("Could not request results from Google Speech Recognition service; {0}".format(e))
          pass
    print("😉 STT done! 😉")  
    
    chked=""
    line = kss.split_sentences(txt)
    for i in range(len(line)):
      #if i%20==0:
      #  print(str(i)+"/"+str(len(line))+" lines done")
      line[i] = spell_checker.check(line[i])[2]
      chked += "".join(line[i])
      chked += ". "
    print("😉 Spell Chekced! 😉")
    
    return chked

/content/drive/Shareddrives/KPMG_Ideation/hanspell


In [ ]:
%cd /content/drive/Shareddrives/KPMG_Ideation/data
file_name='audio_only_2.m4a'
chked = STT_loop(file_name)

/content/drive/Shareddrives/KPMG_Ideation/data
😉 STT done! 😉
😉 Spell Chekced! 😉


In [ ]:
# np.array(['w'])
%cd /content/drive/Shareddrives/KPMG_Ideation/data
t=open('./texts/'+file_name.split('.')[0] + '.txt', "w")
n=t.write(chked)
t.close()

/content/drive/Shareddrives/KPMG_Ideation/data


In [ ]:
chked[:1000]

'일단은 수연이 잘 된다는 거 다 형님이나 가셨어요. 아 이거 한 번 또 다른 분 생각은 조금 더 중요한 얘기하기 전에 뭐 하나 물어 보고 싶은 게 있는데 그 혹시 화면 공유 한번 다시 해 주실 수 있을까요. 어떤 거예요. 화면 공유 다시 한 번만 그 아까 자연의 소리한테 버튼 세 가지 있어 가지고 세제를 써 가지고 그 그 세 개를 종합을 해 가지고서 머리를 부른다고 하셨잖아요. 어떻게 하는 거니 처리를 해도 자연어 처리해서 전체에 회의록을 있고 그중에서 제일 중요하다고 판단되는 일정을 하나 빼놓은 건가요. 네 이게 법 틀어 색스 링크 넥스링크 다 자기들만의 알고리즘으로 이제 자기네들이 생각했을 때 가장 중요한 문장을 뽑아내는 건데요. 사실 저희가 트라제 유명하긴 한데 버트가 무조건 답이라고 할 순 없잖아요. 그렇기 때문에 앙상블 모양을 만든 이유는 이 중에서 이 세계의 대표적인 자연어 처리 모델이 전북이 문장은 중요하다 하라고 뽑은 문자 있을 거 아니에요. 공통되는 뽑아낸 게 이거입니다. 뽑아낼 때 그동안 하는 건지 혹시 이게 업체 같은 경우에는 저기 저번에 승현 님 저랑 잠깐 얘기했듯이 얘가 정확히 어떤 알고리즘으로 뽑았는지 그런 거 인정 그래도 어떤 일정에 규칙은 있어요. 렉스 링크랑 텍스트 랭크 같은 경우에는요 앞서 말씀드렸던 얘랑 되게 비슷해요. 구글에 페이지랭크 나오는 알고리즘 있는데 얘가 이제 앞에서 무슨 무슨 링크라고 하잖아요. 네가 보통 1년 빈도수라는 단어 길이로 이제 랭크 레벨 쉰다고 했잖아요. 이런 식으로 어떤 자기네들 만에 일정에 위치는 있긴 있어요. 그래서 제가 알기로는 텍스트 랭크 랑이 KR 월드 랭크 랑 비슷한 알고리즘을 써서 아 마이 테크 월드 월드뱅크에서 나온 주유소 있잖아요. 그 키워드가 포함된 문장이라든지 그런 식으로 안 갖고 볼거리 키워드를 단순히 우리의 뭐지 그 영향을 위해 사용하는 게 아니라 프로그램도 알고리즘 지도 보면서 사용하는 거예요. 거기서 이제 추운데 중에서 뭐 다른 사람 아닌가 아니면 뭐 때문에 그런데 같은 

In [ ]:
chked[-1000:]

'일에는 보여 드릴 수 있도록 좀 노력 한번 해 보도록 하겠습니다. 네 거 같아서 손님이랑 저는 좀 더 하고 그리고 민찬 님이랑 주연 님을 그 백신 쪽에 좀 쉬면해 주시는 걸로 가면은 주차하기가 그렇다면 회계 관리 쪽은 Air 아래쪽에 어떻게 해야 될까요. 그것도 이제 그 심적으로 구현을 하고 해 보는 프로세스에 계신 걸까요. 혹시 그거에 대해서 조금 벌더니 있으시면은 왜냐면 지금 그 기술적 구라를 두 개를 하셔야 되는 거잖아요. 그렇게 되면 이상적으로 하고 정도나 그 정도는 저도 할 수 있으니까 만약에 도움이 필요하시다 쓰시면 됩니다. 그리고 그 회의록을 어떤 식으로 했지 또 이제 구현하면서 가야겠네요. 형님이랑 가영 님은 이번 주까지 너무 잘 작동해 주셔서 그러면 이것만 하나만 부탁드릴게요 형님이 보내 주셨던 신 관련이 아이디어 제안서 있잖아요. 이거를 회의록 버전으로 가능하실까요. 저희 둘이서 그거를 정리하면 될 거 같은데 다음 주까지 충분히 가능할 거 같고 대한 찬성을 써달라는 말씀이시죠 또 저거를 회의록 일단 기술적으로 구현하는 것 먼저 중심을 볼게요 서리지 언니는요 다음에 아까 그 말 했어 다 같이 그냥 조금만 조금 뭐 이렇게 뭐 하다가 좀 남는 시간 있으면은 어 어 이쪽으로 할 수도 있지 않을까 언제까지나 새로운 아이디어에서 오픈해 있는 걸로 하겠습니다. 많았으니까 그 정도로 생각해 주시면 될 거 같고 그러면 일단은 다음 주 목요일 날에 생각을 하고 있는데 혹시 너무 빠를까요. 22일 21일 22일이 정도야 21일 혹은 22일에 더 선호하는 날짜가 있으시거나 이날 안된다라는 분 있으면은 말 21일 좋아요. 어떠세요. 24일까지 밤이니까 혹여나 22 23일 갑작스럽게 만나야 될 수도 있으니까 21일이 좀 더 그러면 다음 주 목요일 21일 7시에 뵙도록 하고 그전까지 이제 아까 말씀드렸듯이 둘러 다녀서 조금 기술적인 면에서 좀 더 신경 써 주시면 될 거 같고 남은 두 분은 이 제안서 작성해 조금 신경 써 주시면 될 거 같습니다. 하셨어요. 고맙습니다.

In [ ]:
#@title 자연어 처리 모듈
okt = Okt()
class Text(): 
    def __init__(self, text):
        text = re.sub("'", ' ', text)
        paragraphs = text.split('\n')
        self.text = text
        self.paragraphs = [i for i in paragraphs if i]
        self.counts = len(self.paragraphs)
        self.docs = [kss.split_sentences(paragraph) for paragraph in paragraphs if kss.split_sentences(paragraph)]
        self.newtext = deepcopy(self.text)
        print("TEXT")

    def findall(self, p, s):
        i = s.find(p)
        while i != -1:
            yield i
            i = s.find(p, i + 1)
      
    def countMatcher(self, sentences, paragraph_no):
        paragraph = self.docs[paragraph_no]
        total_no = len(paragraph)
        vec = [0] * total_no
        
        for idx, candidate in enumerate(paragraph):
            for sentence in sentences:
                if sentence[:4] in candidate:
                    vec[idx] += 1
        return vec


class Highlight(Text):
    def __init__(self, text):
        super().__init__(text)
        print("Highlight")
        wordrank_extractor = KRWordRank(min_count=3, max_length=10)
        self.keywords, rank, graph = wordrank_extractor.extract(self.paragraphs)
        self.path = "/content/drive/Shareddrives/KPMG_Ideation/OpenInformationExtraction/frameBERT"
        p = []
        kw = []
        for k, v in self.keywords.items():
            p.append(okt.pos(k))
            kw.append(k)
        words = self.text.split(' ')
        s = set()
        keylist = [word for i in kw for word in words if i in word]
        keylist = [i for i in keylist if len(i)>2]
        for i in keylist:
            if len(i)>2:
              s.add(i)
        # print("KEYLIST: ",keylist)

        p = [okt.pos(word) for word in s]
        self.s = set()
        for idx in range(len(p)):
            ls = p[idx]
            for tags in ls:
              word,tag = tags
              if tag == "Noun":
                if len(word)>=2:
                  self.s.add(word)
        self.keys = []
        for temp in self.s:
            self.keys.append(" " + str(temp))
        print("KEYWORDS: ", self.keys)

    def add_tags_conj(self, txt):
        conj = '그리고, 그런데, 그러나, 그래도, 그래서, 또는, 및, 즉, 게다가, 따라서, 때문에, 아니면, 왜냐하면, 단, 오히려, 비록, 예를 들어, 반면에, 하지만, 그렇다면, 바로, 이에 대해'
        conj = conj.replace("'", "")
        self.candidates = conj.split(",")
        self.newtext = deepcopy(txt)
        self.idx = [(i, i + len(candidate)) for candidate in self.candidates for i in
                        self.findall(candidate, txt)]
        for i in range(len(self.idx)):
            try:
                self.idx = [(start, start + len(candidate)) for candidate in self.candidates for start in
                            self.findall(candidate, self.newtext)]
                word = self.newtext[self.idx[i][0]:self.idx[i][1]]
                self.newtext = word.join([self.newtext[:self.idx[i][0]], self.newtext[self.idx[i][1]:]])
            except:
                pass
        return self.newtext

In [ ]:
class Summarize(Highlight):
    def __init__(self, text, paragraph_no):
      super().__init__(text)
      print("length of paragraphs ",len(self.paragraphs))
      self.txt = self.paragraphs[paragraph_no]
      self.paragraph_no = paragraph_no

    # 이부분 fix 
    # def summarize(self):
    #     url = "https://api.smrzr.io/v1/summarize?num_sentences=5&algorithm=kmeans"
    #     headers = {
    #         'content-type': 'raw/text',
    #         'origin': 'https://smrzr.io',
    #         'referer': 'https://smrzr.io/',
    #         'sec-fetch-dest': 'empty',
    #         'sec-fetch-mode': 'cors',
    #         'sec-fetch-site': 'same-site',
    #         "user-agent": "Mozilla/5.0"
    #     }
    #     resp = requests.post(url, headers=headers, data= self.txt.encode('utf-8'))
    #     assert resp.status_code == 200
    #     summary = resp.json()['summary']
    #     temp = summary.split('\n')
    #     print("BERT: ", temp)
    #     return temp



    def summarizeTextRank(self):
        summary = summarize(self.txt, ratio=0.3).split("\n")
        print("Textrank: ",summary)
        return summary


    def summarizeLexRank(self):
        lr = LexRank()
        lr.summarize(self.txt)
        summaries = lr.probe(0.3)
        print("Lexrank: ",summaries)
        return summaries

    def ensembleSummarize(self):
        # a = np.array(self.countMatcher(self.summarize(), self.paragraph_no))
        
        try:
          b = np.array(self.countMatcher(self.summarizeLexRank(), self.paragraph_no))
        except:
          b = np.zeros_like(a)
        c = np.array(self.countMatcher(self.summarizeTextRank(),self.paragraph_no))
        result= b+c
        i, = np.where(result == max(result))
        txt, index = self.docs[self.paragraph_no][i[0]], i[0]
        return txt, index

In [ ]:
result = chked
high = Highlight(result)

TEXT
Highlight
KEYWORDS:  [' 보시', ' 위해', ' 구형', ' 결정', ' 미국', ' 도출', ' 상호작용', ' 직업', ' 인터뷰', ' 대해', ' 그것', ' 변수', ' 스토리', ' 모델링', ' 토픽', ' 저희', ' 확률', ' 세분', ' 다음', ' 로만', ' 사람과', ' 파악', ' 슬라이드', ' 드릴', ' 사용', ' 근거', ' 공급', ' 대답', ' 텍스트', ' 해결', ' 노력', ' 모형', ' 대충', ' 사실', ' 시간', ' 전체', ' 별로', ' 우리', ' 따라서', ' 로서', ' 이상', ' 자기', ' 감소', ' 라라', ' 얘기', ' 설명', ' 제이', ' 데이터', ' 기술', ' 아이디어', ' 단계', ' 클로버', ' 월드뱅크', ' 궁금', ' 거가', ' 결과', ' 적용', ' 의사결정', ' 공휴일', ' 과제', ' 통해', ' 가면', ' 위험', ' 과정', ' 페이지랭크', ' 수치', ' 확률분포', ' 넥스', ' 표현', ' 실제', ' 동안', ' 언제', ' 워드', ' 멀티', ' 사람', ' 나이', ' 일자', ' 지금', ' 때문', ' 시각', ' 모델', ' 치사', ' 건가', ' 민찬', ' 추가', ' 일정', ' 이닝', ' 정도', ' 저번', ' 해당', ' 강사', ' 연령', ' 제대로', ' 정보', ' 일어나고', ' 대가', ' 언니', ' 혹시', ' 가치', ' 얼마', ' 퍼센트', ' 상태', ' 구현', ' 수준', ' 업데이트', ' 정말', ' 영향', ' 아이디', ' 주제', ' 보조', ' 기존', ' 당장', ' 일단', ' 질문', ' 대요', ' 라인', ' 치평', ' 서도', ' 이야기', ' 이제', ' 건전성', ' 보고서', ' 이해인', ' 가능', ' 이용', ' 형식', ' 이면', ' 반영', ' 선행', ' 레이', ' 무증상', ' 아이', ' 분식회계', ' 선호

In [ ]:
summarizer = Summarize(chked, 0)
sum, id = summarizer.ensembleSummarize()
print("summarized ",sum)

TEXT
Highlight
KEYWORDS:  [' 보시', ' 위해', ' 구형', ' 결정', ' 미국', ' 도출', ' 상호작용', ' 직업', ' 인터뷰', ' 대해', ' 그것', ' 변수', ' 스토리', ' 모델링', ' 토픽', ' 저희', ' 확률', ' 세분', ' 다음', ' 로만', ' 사람과', ' 파악', ' 슬라이드', ' 드릴', ' 사용', ' 근거', ' 공급', ' 대답', ' 텍스트', ' 해결', ' 노력', ' 모형', ' 대충', ' 사실', ' 시간', ' 전체', ' 별로', ' 우리', ' 따라서', ' 로서', ' 이상', ' 자기', ' 감소', ' 라라', ' 얘기', ' 설명', ' 제이', ' 데이터', ' 기술', ' 아이디어', ' 단계', ' 클로버', ' 월드뱅크', ' 궁금', ' 거가', ' 결과', ' 적용', ' 의사결정', ' 공휴일', ' 과제', ' 통해', ' 가면', ' 위험', ' 과정', ' 페이지랭크', ' 수치', ' 확률분포', ' 넥스', ' 표현', ' 실제', ' 동안', ' 언제', ' 워드', ' 멀티', ' 사람', ' 나이', ' 일자', ' 지금', ' 때문', ' 시각', ' 모델', ' 치사', ' 건가', ' 민찬', ' 추가', ' 일정', ' 이닝', ' 정도', ' 저번', ' 해당', ' 강사', ' 연령', ' 제대로', ' 정보', ' 일어나고', ' 대가', ' 언니', ' 혹시', ' 가치', ' 얼마', ' 퍼센트', ' 상태', ' 구현', ' 수준', ' 업데이트', ' 정말', ' 영향', ' 아이디', ' 주제', ' 보조', ' 기존', ' 당장', ' 일단', ' 질문', ' 대요', ' 라인', ' 치평', ' 서도', ' 이야기', ' 이제', ' 건전성', ' 보고서', ' 이해인', ' 가능', ' 이용', ' 형식', ' 이면', ' 반영', ' 선행', ' 레이', ' 무증상', ' 아이', ' 분식회계', ' 선호

In [ ]:
sum

'그래서 만약에 주연이 힘들다면 보는 게 어떨까 그리고 말씀하셨던 말씀하셨던 그 일정은 그것도 그렇게 어렵진 않을 것 같아서요.'

In [ ]:
# 놀라운 결과: Lexrank, Textrank 결과 자세히 보면 영어가 인식되어있음ㅎ

- 사용자 인식(speaker identification)이 됐으면 좋겠다 -- clova NOTE 사용시 해결
>무료 api는 supervised만 있는 듯

- 기본 웹프레임워크 만들기
> 아웃풋 어떤 모양일지?

- BERT summarization 모델 추가?
> 기존의 majority voting ensemble대신 그냥 세 모델 결과 다 합칠지? 논의 필요

  > 모델당 뽑는 요약본이 다 다르기 때문에 각 모델별 10%씩 합친다면 약 30% 요약

    >이 경우 소연 자연어 처리 모듈 조금 수정하면 됨




